# Trabalho 5


### 1) Resumo das Técnicas de Imputação de Variáveis

- **Média**: adequada para variáveis numéricas com distribuição simétrica e poucos outliers.
- **Mediana**: preferida quando há outliers, pois é mais robusta.
- **Moda**: usada para variáveis categóricas.
- **Valor Fixo**: deve ser usado com cautela para não distorcer os dados.
- **KNN**: considera a similaridade entre registros, útil quando há padrões não lineares.
- **Regressão**: utiliza uma variável relacionada para prever o valor ausente.
- **Múltipla Imputação**: gera várias versões dos dados e combina os resultados, sendo ideal para inferência estatística.


###2)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

dados = pd.read_csv('/content/PimaIndiansDiabetes2.csv')

print("Valores ausentes por coluna:")
print(dados.isnull().sum())

Valores ausentes por coluna:
Unnamed: 0      0
Diabetes        0
pregnant        0
glucose         5
pressure       35
triceps       227
insulin       374
mass           11
pedigree        0
age             0
dtype: int64


In [ ]:
def verificar_outliers_em_variaveis(dados, colunas=None):
    if colunas is None:
        colunas = ["glucose", "pressure", "mass", "insulin", "triceps"]

    resultados = {}

    for coluna in colunas:
        serie = dados[coluna].dropna()

        # Cálculo do IQR
        q1 = serie.quantile(0.25)
        q3 = serie.quantile(0.75)
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr

        # Identifica outliers
        outliers = serie[(serie < limite_inferior) | (serie > limite_superior)]
        n_total = len(serie)
        n_outliers = len(outliers)
        perc = (n_outliers / n_total) * 100

        resultados[coluna] = {"total": n_total, "outliers": n_outliers, "percentual": perc, "recomendado": "mediana" if n_outliers > 0 else "média"}

        print(f"{coluna}: {n_outliers} outliers de {n_total} valores válidos "
              f"({perc:.1f}% dos dados) → sugerido: {resultados[coluna]['recomendado']}")

    return resultados

resultados = verificar_outliers_em_variaveis(dados)

glucose: 0 outliers de 763 valores válidos (0.0% dos dados) → sugerido: média
pressure: 14 outliers de 733 valores válidos (1.9% dos dados) → sugerido: mediana
mass: 8 outliers de 757 valores válidos (1.1% dos dados) → sugerido: mediana
insulin: 24 outliers de 394 valores válidos (6.1% dos dados) → sugerido: mediana
triceps: 3 outliers de 541 valores válidos (0.6% dos dados) → sugerido: mediana


In [ ]:
dados_imputados = dados.copy()

# Imputação pela mediana
imputador_mediana = SimpleImputer(strategy="median")
dados_imputados[["pressure", "mass", "insulin"]] = imputador_mediana.fit_transform(dados[["pressure", "mass", "insulin"]])

# Imputação pela média
imputador_media = SimpleImputer(strategy="mean")
dados_imputados[["glucose"]] = imputador_media.fit_transform(dados[["glucose"]])

# Imputação por regressão para dobra do tríceps com base em mass
triceps_dados = dados[["triceps"]].copy()
triceps_dados["mass"] = dados_imputados["mass"]
treino_triceps = triceps_dados.dropna()
prever_triceps = triceps_dados[triceps_dados["triceps"].isna()]
modelo_triceps = LinearRegression()
modelo_triceps.fit(treino_triceps[["mass"]], treino_triceps["triceps"])
dados_imputados.loc[dados["triceps"].isna(), "triceps"] = modelo_triceps.predict(prever_triceps[["mass"]])

# Estatísticas antes e depois da imputação
antes = dados.describe()
depois = dados_imputados.describe()

print("\nEstatísticas antes da imputação:")
print(antes)

print("\nEstatísticas depois da imputação:")
print(depois)



Estatísticas antes da imputação:
       Unnamed: 0    Diabetes    pregnant     glucose    pressure     triceps  \
count  768.000000  768.000000  768.000000  763.000000  733.000000  541.000000   
mean   384.500000    0.348958    3.845052  121.686763   72.405184   29.153420   
std    221.846794    0.476951    3.369578   30.535641   12.382158   10.476982   
min      1.000000    0.000000    0.000000   44.000000   24.000000    7.000000   
25%    192.750000    0.000000    1.000000   99.000000   64.000000   22.000000   
50%    384.500000    0.000000    3.000000  117.000000   72.000000   29.000000   
75%    576.250000    1.000000    6.000000  141.000000   80.000000   36.000000   
max    768.000000    1.000000   17.000000  199.000000  122.000000   99.000000   

          insulin        mass     pedigree         age  
count  394.000000  757.000000   768.000000  768.000000  
mean   155.548223   32.457464   428.235091   33.240885  
std    118.775855    6.924988   340.485655   11.760232  
min     

In [ ]:
variaveis_para_comparar = ["glucose", "pressure", "triceps", "insulin", "mass"]

for variavel in variaveis_para_comparar:
    plt.figure(figsize=(10, 4))
    sns.kdeplot(dados[variavel], label='Antes', fill=True)
    sns.kdeplot(dados_imputados[variavel], label='Depois', fill=True)
    plt.title(f"Distribuição da variável \"{variavel}\" antes e depois da imputação")
    plt.legend()
    plt.show()


**glucose (concentração de glicose):**

* Não há outliers
* Técnica sugerida: média
* Justificativa: Como não foram detectados outliers, a média é apropriada para representar a tendência central, pois mantém a variabilidade natural dos dados.

---

**pressure (pressão arterial):**

* Há outliers
* Técnica sugerida: mediana
* Justificativa: Apesar da distribuição relativamente simétrica, a presença de alguns outliers recomenda o uso da mediana, que é mais robusta e evita distorções causadas por valores extremos.

---

**mass (IMC):**

* Há outliers
* Técnica sugerida: mediana
* Justificativa: Presença de outliers justifica o uso da mediana para minimizar o impacto desses valores extremos e preservar a tendência central.

---

**insulin (nível de insulina):**

* Há outliers
* Técnica sugerida: mediana
* Justificativa: Com alta proporção de valores ausentes (\~49%) e diversos outliers, a mediana é mais segura para evitar viés e distorções que um modelo de regressão poderia causar com dados escassos.

---

**triceps (espessura da prega cutânea do tríceps):**

* Há outliers
* Técnica sugerida: mediana com regressão linear usando mass
* Justificativa: Embora poucos outliers estejam presentes, existe uma correlação fisiológica entre o IMC e a espessura da prega cutânea, permitindo prever valores ausentes com um modelo de regressão linear baseado em mass, garantindo melhor estimativa.

---